# Salesforce/instructblip-vicuna-7b

# Brightness-order

In [ ]:
import torch
from PIL import Image
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from collections import defaultdict
import pandas as pd
import re

# Load the InstructBLIP model and processor
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-vicuna-7b",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to("cuda:0")

# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Brightness/{row['path']}"
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Contrast-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Contrast/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Defocus-blur-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Defocus-blur/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Elastic-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Elastic/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Fog-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Fog/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Frost-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Frost/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Gaussian-noise-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Gaussian-noise/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Impulse-Noise-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Impulse-noise/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# JPEG-compresion-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/JPEG-compression/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Pixelate-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Pixelate/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Rain-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Rain/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Saturation-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Saturation/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Shot-noise-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Shot-noise/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Snow-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Snow/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Spatter-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Spatter/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Speckle-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Speckle-noise/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Zoom-blur-Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Zoom-Blur/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


## Motion-blur Order

In [ ]:
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/OrderCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "order":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Motion-blur/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

## Brightness - Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Brightness/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Contrast Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Contrast/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Defocus-blur Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Defocus-blur/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Elastic Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Elastic/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Fog Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Fog/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Frost Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Frost/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Gaussian-noise Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Gaussian-noise/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Impulse-noise Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Impulse-noise/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# JPEG-compresion-Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/JPEG-compression/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Pixelate-Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Pixelate/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Rain trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Rain/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Saturation Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Saturation/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Shot-noise Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Shot-noise/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Snow Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Snow/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Spatter Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Spatter/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Speckle Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Speckle-noise/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Zoom Blur Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Zoom-Blur/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Motion Blur Trick

In [ ]:
# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/TrickCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "trick":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"DENOISED DARE DRUNET (MAIN)/Motion-blur/{row['path']}"
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found for Question ID {row['id']} at path: {image_path}. Skipping...")
        continue
    except Exception as e:
        print(f"Error loading image for Question ID {row['id']} at path: {image_path}. Error: {e}. Skipping...")
        continue

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")